# Song Recommender  

## importing libraries

In [1]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import random
import pprint
import pickle
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from IPython.display import IFrame
import config

## Accessing the Spotify API

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.c_id,
                                                           client_secret=config.c_se))

## Loading data files

In [3]:
df = pd.read_csv('clustered_songs_df.csv')
hot_songs = pd.read_csv('top_100_songs.csv')
with open("scaler.pickle", "rb") as f: 
        scaler = pickle.load(f)
with open("kmeans3.pickle", "rb") as f: 
        kmeans3 = pickle.load(f) 

## Song recommender 

In [18]:
for song in hot_songs[["name","artist"]].iterrows():
    user_input = input("Write down a song title: ")
    if user_input.lower() in list(hot_songs["name"]):
        random_song = random.choice(hot_songs["name"])
        song_artist = hot_songs.loc[hot_songs["name"] == random_song, "artist"].values[0]
        print("This is a hot song")
        print("You may also like this song: ", random_song, "from", song_artist)
        print("Thanks for using the music recommender.")
        break

    else:
        
        results = sp.search(q= user_input,limit=1,market="GB")
        track_id=results['tracks']['items'][0]['external_urls']["spotify"][31:]
        results["tracks"]["items"][0]["uri"]
        input_song = sp.audio_features(results["tracks"]["items"][0]["uri"])[0]
        input_song = pd.DataFrame([input_song])
        input_song_num = input_song.select_dtypes(np.number)
        input_song_sc = scaler.transform(input_song_num)
        song_trans_df = pd.DataFrame(input_song_sc, columns = input_song_num.columns)
        user_song_cluster = kmeans3.predict(song_trans_df)[0]
        recommended_song = df[df["cluster"] == user_song_cluster].sample()
        recommended_song_id = recommended_song["id"].iat[0]
        print("You might also like the song: " + recommended_song["song_title"].iat[0] + " by " + recommended_song["artist"].iat[0] + ".") 
        
        break

Write down a song title: fly me to the moon
You might also like the song: Heartbreak Hotel by Elvis Presley.


In [19]:
display(IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",))

display(IFrame(src="https://open.spotify.com/embed/track/"+recommended_song_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",))